# Federated Learning Model Poisoning Attack Simulation

In [137]:
!pip install -q flwr[simulation] torch torchvision matplotlib opacus

from collections import OrderedDict
from typing import List, Tuple, Optional, Callable, Dict
from matplotlib import pyplot as plt
import certifi
import ssl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import CIFAR10
import flwr as fl
from flwr.common import Parameters, Scalar, FitRes, Metrics
import os
from opacus import PrivacyEngine
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
from tqdm import tqdm

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # fixed something with matplotlib killing the kernel
ssl._create_default_https_context = ssl._create_unverified_context  # fixed something with the ssl certificate of the dataset
DEVICE = torch.device("cpu")  # "cpu" -> train in cpu | "cuda" -> train in gpu

In [138]:
from logging import WARNING # we need those imports to implement the strategy class
from typing import Callable, Dict, List, Optional, Tuple, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy

from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg, aggregate_krum
from flwr.server.strategy.strategy import Strategy

In [139]:
CLASSES = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck") # classes of the example dataset

NUM_CLIENTS = 10 # this is the number of devices participating in the federated learning

BATCH_SIZE = 32 # this is the size of a mini-batch for the training of a CNN using SGD

In [140]:
def load_datasets(): # download and transform cifar-10
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10% validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets() # load the datasets

Files already downloaded and verified
Files already downloaded and verified


In [141]:
class Net(nn.Module): # here we create a neural network class that inherits from nn.Module
    def __init__(self) -> None: # we have to implement only two functions, the constructor and the forward pass
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [142]:
def get_target_delta(data_size: int) -> float: # this is the δ for the (ε, δ) differential privacy
    den = 1
    while data_size // den >= 1:
        den *= 10
    return 1 / den

In [143]:
def train(net, trainloader, epochs: int, nm, mgn, eps, vbatch_size, state_dict, verbose=False): # we need a train and a test function that our clients will be using
    # assert(vbatch_size%BATCH_SIZE==0)
    n_acc_steps = int(vbatch_size / BATCH_SIZE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    alphas = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)) # rdp orders
    delta = get_target_delta(len(trainloader.dataset)) # dp δ
    sample_rate = BATCH_SIZE / len(trainloader.dataset)

    privacy_engine = PrivacyEngine()
    net, optimizer, trainloader = privacy_engine.make_private(
        module=net,
        optimizer=optimizer, # here we attach the optimizer with the DP engine so that in every stochastic gradient descent step it adds noise + gradient clipping + etc!
        data_loader=trainloader,
        noise_multiplier=nm,
        max_grad_norm=mgn,
    )

    if state_dict is not None:
      privacy_engine.load_state_dict(state_dict)

    for epoch in range(epochs):

        correct, total, epoch_loss, ctr = 0, 0, 0.0, 0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images) # do the forward pass
            loss = criterion(outputs, labels) # calculate the loss function
            loss.backward() # calculate the gradients of the loss function


            optimizer.step()  # the optimizer is now a dp-optimizer and this will be a real step or a virtual step depending on the dp state!
            optimizer.zero_grad()

            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            ctr = ctr + 1
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total

        if verbose: # log information
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [144]:
def get_parameters(net) -> List[np.ndarray]: # we need these two function because this way flower knows how
    return [val.cpu().numpy() for _, val in net.state_dict().items()] # to serialize/deserialize data

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [145]:
class FlowerClient(fl.client.NumPyClient): # here we define the FlowerClient. We have to implement only the following 3 methods
    def __init__(self, net, trainloader, valloader): # of course we can customize these methods as we like
        self.net = net                            # we can add extra member variables as well
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1, mgn=1.0, nm=1.2, eps=0.0, vbatch_size=128, state_dict= None)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [146]:
class BenignFlowerClient(FlowerClient):
    def __init__(self, net, trainloader, valloader):
        super().__init__(net, trainloader, valloader)

    def fit(self, parameters, config):
        #print("Benign Client Got Selected", flush=True)
        parameters, num_examples, _ = super().fit(parameters, config)
        return parameters, num_examples, {"intention": "BENIGN"}

class MaliciousFlowerClient(FlowerClient):
    def __init__(self, net, trainloader, valloader):
        super().__init__(net, trainloader, valloader)

    def fit(self, parameters, config):
        #print("Malicious Client Got Selected", flush=True)
        parameters, num_examples, _ = super().fit(parameters, config)
        return parameters, num_examples, {"intention": "MALICIOUS"}

In [147]:
def client_fn(cid: str) -> FlowerClient: # this is a factory function (factory desing pattern)
    net = Net().to(DEVICE) # flower calls this function to create FlowerClients on demand (this way it uses less memory)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    if int(cid) <= NUM_CLIENTS * 0.2:    # 20 - 80 --> malicious - benign
        return MaliciousFlowerClient(net, trainloader, valloader)
    else:
        return BenignFlowerClient(net, trainloader, valloader)

In [148]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics: # we pass this function as an argument to the Strategy
    # calculate accuracy
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # return it as a dictionary
    return {"accuracy": sum(accuracies) / sum(examples)}

In [149]:
class AttackSimulationStrategy(fl.server.strategy.FedAvg): # we inherit from FedAvg strategy and change only what we need
    def __init__(self, *, fraction_fit: float = 1.0, fraction_evaluate: float = 1.0, min_fit_clients: int = 2,
        min_evaluate_clients: int = 2, min_available_clients: int = 2, num_malicious_clients: int = 2,
        num_clients_to_keep: int = 0, evaluate_fn: Optional[Callable[[int, NDArrays, Dict[str, Scalar]],
        Optional[Tuple[float, Dict[str, Scalar]]],]] = None, on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None, accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None, fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        perturbationVector: str, adversaryKnowledge: str
    ) -> None:
        super().__init__(
            fraction_fit=fraction_fit, fraction_evaluate=fraction_evaluate, min_fit_clients=min_fit_clients,
            min_evaluate_clients=min_evaluate_clients, min_available_clients=min_available_clients, evaluate_fn=evaluate_fn,
            on_fit_config_fn=on_fit_config_fn, on_evaluate_config_fn=on_evaluate_config_fn, accept_failures=accept_failures,
            initial_parameters=initial_parameters, fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
            evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
        )
        self.num_malicious_clients = num_malicious_clients
        self.num_clients_to_keep = num_clients_to_keep
        self.perturbationVector = perturbationVector
        self.adversaryKnowledge = adversaryKnowledge
        self.max_epsilon = 0.0 # this is the global privacy budget

    def __repr__(self) -> str:
        rep = f"AttackSimulationStrategy(accept_failures={self.accept_failures})"
        return rep

    def aggregate_fit(self, server_round: int, results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:

        if not results: # boilerplate code to handle exceptions
            return None, {}
        if not self.accept_failures and failures:
            return None, {}


        accepted_results = [] # get the privacy budget of each client
        disconnect_clients = []
        epsilons = []
        for c, r in results:
            if r.metrics["accept"]:
                accepted_results.append([c, r])
                epsilons.append(r.metrics["epsilon"])
            else:
                disconnect_clients.append(c)

        #if disconnect_clients: # disconnect clients if needed
            #shutdown(disconnect_clients)
        results = accepted_results
        if epsilons:
            self.max_epsilon = max(self.max_epsilon, max(epsilons))
        print(f"Privacy budget ε at round {server_round}: {self.max_epsilon}")
        # Call aggregate_evaluate from base class (FedAvg)
        return super().aggregate_fit(server_round, results, failures)

        # convert results
        total_weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) for _, fit_res in results]

        """
        #############################################################################
        #########################----SIMULATE THE ATTACK----#########################
        #############################################################################
        benign_weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) for _, fit_res in results
                         if fit_res.metrics["intention"] == "BENIGN"]
        if self.adversaryKnowledge == "agr-only" or self.adversaryKnowledge == "agnostic":
            malicious_weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) for _, fit_res in results
                                if fit_res.metrics["intention"] == "MALICIOUS"]
            best_parameters = aggregate(total_weights_results) # malicious_weights_results
        else:
            best_parameters = aggregate(total_weights_results)

        # so far we have [1]: all clients parameters, [2]: benign clients parameters (seperate),
        #                [3]: malicious clients parameters (seperate), [4]: best_parameters (based on the adversary's knowledge)
        # no we have to optimize the model poisoning attack based on the adversary's knowledge and the perturbation vector

        # calculate the perturbation vector
        if self.perturbationVector == "InverseSign":
            perturbationVector = [-np.sign(best_parameter) for best_parameter in best_parameters]
        if self.perturbationVector == "InverseStd":
            parameters = [parameters for parameters, _ in total_weights_results]
            perturbationVector = []

            for parameterIdx in range(len(parameters[0])):
                arr = []
                for clientIdx in range(len(parameters)):
                    arr.append(parameters[clientIdx][parameterIdx])
                numpyArr = np.stack(arr)
                perturbationVector.append(-np.std(numpyArr, axis=0))

        # optimize for gamma
        gamma_init = 100.0
        gamma = gamma_init
        gamma_succ = 0.0
        t = 0.000001 # threshold
        step = gamma_init

        while True: # this is a c++ do-while loop emulation
            malicious_parameters = []
            for idx in range(len(perturbationVector)):
                malicious_parameters.append(best_parameters[idx] + gamma*perturbationVector[idx])

            new_malicious_weights_results = [(malicious_parameters, num_examples) for _, num_examples in malicious_weights_results]
            new_total_weights_results = new_malicious_weights_results + benign_weights_results

            are_equal = True
            krum_parameters = aggregate_krum(new_total_weights_results, self.num_malicious_clients, self.num_clients_to_keep)
            for idx in range(len(krum_parameters)):
                if np.array_equal(malicious_parameters[idx], krum_parameters[idx]) == False:
                    are_equal = False
                    break

            if are_equal:
                gamma_succ = gamma
                gamma = gamma + step/2
            else:
                gamma = gamma - step/2
            step = step/2

            if abs(gamma_succ - gamma) < t:
                break

            print(gamma, gamma_succ)

        malicious_parameters = []
        for idx in range(len(perturbationVector)):
            malicious_parameters.append(best_parameters[idx] + gamma_succ*perturbationVector[idx])
        new_malicious_weights_results = [(malicious_parameters, num_examples) for _, num_examples in malicious_weights_results]
        total_weights_results = new_malicious_weights_results + benign_weights_results
        #############################################################################
        #############################################################################
        #############################################################################
        """

        # calculate AGR
        # parameters_aggregated = ndarrays_to_parameters(aggregate_krum( # this thing here is for krum-AGR
        #    total_weights_results, self.num_malicious_clients, self.num_clients_to_keep))

        # calculate Non Attack AGR
        parameters_aggregated = ndarrays_to_parameters(aggregate(total_weights_results))

        # boilerplate code if aggregation metrics are provided
        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)
        elif server_round == 1:  # Only log this warning once
            log(WARNING, "No fit_metrics_aggregation_fn provided")

        return parameters_aggregated, metrics_aggregated



In [150]:
# create a strategy
strategy = AttackSimulationStrategy(fraction_fit=0.8, fraction_evaluate=0.5, min_fit_clients=5,
    min_evaluate_clients=5, min_available_clients=10, perturbationVector="InverseStd", adversaryKnowledge="agr-only",
    evaluate_metrics_aggregation_fn=weighted_average,) # <-- pass the metric aggregation function. This function will be called
                                                       # in every federated learning round for evaluation (it aggregates the
                                                       # client-side evaluation metrics in the server)

# start simulation
fl.simulation.start_simulation(
    client_fn=client_fn, # out factory function
    num_clients=NUM_CLIENTS, # number of clients
    config=fl.server.ServerConfig(num_rounds=5), # number of federated learning rounds
    strategy=strategy, # our attack simulation strategy
    client_resources=None,
)

INFO flwr 2023-09-25 10:08:38,651 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-09-25 10:08:43,658	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-25 10:08:47,434 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3939189964.0, 'memory': 7878379931.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3939189964.0, 'memory': 7878379931.0}
INFO flwr 2023-09-25 10:08:47,442 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-25 10:08:47,449 | app.py:224 | Flower VCE: Resources for each Virtual Clie

History (loss, distributed):
	round 1: 0.07361913843154907
	round 2: 0.07341510095596314
	round 3: 0.07257387256622314
	round 4: 0.07090234088897705
	round 5: 0.06886343116760255
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.11320000000000001), (2, 0.14120000000000002), (3, 0.2008), (4, 0.2052), (5, 0.21200000000000002)]}